In [12]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

import sys
import os

# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath('../src'))

# Import the module

import numpy as np
import pandas as pd
from neural_network.layers.dense_layer import DenseLayer
from neural_network.layers.dense_l2_layer import L2DenseLayer
from neural_network.layers.dropout_layer import DropoutLayer
from neural_network.optimizer import GradientDescent
from neural_network.losses import CrossEntropyLoss
from neural_network.utils import normalize_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name 'DenseL2Layer' from 'neural_network.layers.dense_l2_layer' (/home/isac/Desktop/Untitled Folder 8/machine-learning/src/neural_network/layers/dense_l2_layer.py)

In [2]:

df = pd.read_csv('../data/binary/breast_cancer.csv')

df = df.drop('id', axis=1)
df = df.drop('Unnamed: 32', axis=1)
valid_values = ['M', 'B']
df = df[df['diagnosis'].isin(valid_values)]

df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:

df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

X = df.drop('diagnosis', axis=1).values
y = df['diagnosis'].values

df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
X = normalize_data(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
class NeuralNetwork:
    def __init__(self, layers, loss, optimizer):
        self.layers = layers
        self.loss = loss
        self.optimizer = optimizer
        
    def forward(self, X):
        output = X
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, y_true, y_pred):
        grad = self.loss.backward(y_true, y_pred)
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
    
    def update(self):
        for layer in self.layers:
            params = layer.get_parameters()
            if params:
                grads = layer.get_gradients()
                weights_updated, biases_updated = self.optimizer.update(
                    params['weights'], params['biases'],
                    grads['weights'], grads['biases']
                )
                layer.set_parameters({'weights': weights_updated, 'biases': biases_updated})


In [11]:
input_size = X_train.shape[1]

layers = [
    DenseLayer(input_size=input_size, output_size=16, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseLayer(input_size=16, output_size=8, activation='relu'),
    DropoutLayer(rate=0.2),
    DenseL2Layer(input_size=8, output_size=4, activation='leaky_relu'),
    DenseLayer(input_size=4, output_size=1, activation='sigmoid')
]

# Define loss and optimizer
loss = CrossEntropyLoss()
optimizer = GradientDescent(learning_rate=0.01)

# Initialize the model
model = NeuralNetwork(layers=layers, loss=loss, optimizer=optimizer)


NameError: name 'DenseL2Layer' is not defined

In [7]:
epochs = 400

for epoch in range(epochs):
    # Forward pass
    y_pred = model.forward(X_train)
    
    # Compute loss
    loss_value = loss.calculate(y_train.reshape(-1, 1), y_pred)
    
    # Backward pass
    model.backward(y_train.reshape(-1, 1), y_pred)
    
    # Update parameters
    model.update()
    
    # Compute accuracy
    if (epoch + 1) % 10 == 0:
        predictions = (y_pred > 0.5).astype(int)
        acc = accuracy_score(y_train, predictions)
        print(f'Epoch {epoch + 1}, Loss: {loss_value:.4f}, Accuracy: {acc:.4f}')


Epoch 10, Loss: 0.6792, Accuracy: 0.6198
Epoch 20, Loss: 0.6629, Accuracy: 0.6593
Epoch 30, Loss: 0.6488, Accuracy: 0.6945
Epoch 40, Loss: 0.6369, Accuracy: 0.7165
Epoch 50, Loss: 0.6269, Accuracy: 0.7385
Epoch 60, Loss: 0.6170, Accuracy: 0.7560
Epoch 70, Loss: 0.6071, Accuracy: 0.7648
Epoch 80, Loss: 0.5971, Accuracy: 0.7692
Epoch 90, Loss: 0.5872, Accuracy: 0.7846
Epoch 100, Loss: 0.5770, Accuracy: 0.7890
Epoch 110, Loss: 0.5664, Accuracy: 0.8000
Epoch 120, Loss: 0.5554, Accuracy: 0.8110
Epoch 130, Loss: 0.5442, Accuracy: 0.8154
Epoch 140, Loss: 0.5327, Accuracy: 0.8242
Epoch 150, Loss: 0.5208, Accuracy: 0.8330
Epoch 160, Loss: 0.5092, Accuracy: 0.8396
Epoch 170, Loss: 0.4974, Accuracy: 0.8396
Epoch 180, Loss: 0.4854, Accuracy: 0.8462
Epoch 190, Loss: 0.4735, Accuracy: 0.8484
Epoch 200, Loss: 0.4614, Accuracy: 0.8549
Epoch 210, Loss: 0.4493, Accuracy: 0.8571
Epoch 220, Loss: 0.4378, Accuracy: 0.8637
Epoch 230, Loss: 0.4266, Accuracy: 0.8659
Epoch 240, Loss: 0.4149, Accuracy: 0.8659
E

In [8]:
# Forward pass on test data
y_pred_test = model.forward(X_test)

# Compute loss
test_loss = loss.calculate(y_test.reshape(-1, 1), y_pred_test)

# Compute accuracy
predictions_test = (y_pred_test > 0.5).astype(int)
test_acc = accuracy_score(y_test, predictions_test)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Test Loss: 0.3140, Test Accuracy: 0.8509
